<a href="https://colab.research.google.com/github/zganjei/LLMops-VertexAI/blob/main/llmops_vertexai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install google-cloud-aiplatform
!pip install kfp

Load credentials and relevant Python libraries
NOTE : create a project on GCP and use its name below, in the PROJECT_ID variable

In [ ]:
import google.cloud.aiplatform as aiplatform
from google.colab import auth
from google.auth import default

auth.authenticate_user()
# Verify the current credentials being used
credentials, project = default()
print(project)

PROJECT_ID = "zeinab-llmops-vertexai-452213"
REGION = "us-central1"



Now let's import and initialize VertexAI SDK to be able to interact with VertexAI servers in the cloud

In [ ]:
import vertexai

vertexai.init(project=PROJECT_ID, location=REGION)

I'm going to initialize and use BigQuery data warehouse which is Serverless (so I don't need to manage servers), plus, it uses SQL. SQL is efficient for processing large amounts of data, and is very good for data cleaning and data preparation. Pandas is more used when we have the data locally. SQL is better when data is stored in a data warehouse.

For this project I use StackOverflow public dataset :)
This dataset contains tables for questions, answers, and metadata.

In [ ]:
from google.cloud import bigquery

bq_client = bigquery.Client(project=PROJECT_ID)
print(f"Credentials: {credentials}, Bigquery client initialized for Project: {bq_client.project}")


Credentials: <google.auth.compute_engine.credentials.Credentials object at 0x7b6da911b650>, Bigquery client initialized for Project: zeinab-llmops-vertexai-452213


Let's enable Vertex AI API, Gemini, and cloud storage

In [ ]:
# !gcloud projects list
# !gcloud projects get-iam-policy zeinab-llmops-vertexai-452213
# refresh authentication in Colab
# !gcloud auth application-default login
# !gcloud auth application-default set-quota-project zeinab-llmops-vertexai-452213
# !gcloud services enable --project zeinab-llmops-vertexai-452213 aiplatform.googleapis.com #enable vertex ai
# !gcloud services enable --project zeinab-llmops-vertexai-452213 generativelanguage.googleapis.com #enable gemini
# !gcloud services enable --project zeinab-llmops-vertexai-452213 storage.googleapis.com

!gcloud services list --project zeinab-llmops-vertexai-452213 --enabled | grep -E 'aiplatform|generativelanguage|storage'


aiplatform.googleapis.com           Vertex AI API
bigquerystorage.googleapis.com      BigQuery Storage API
generativelanguage.googleapis.com   Generative Language API
storage-component.googleapis.com    Cloud Storage
storage.googleapis.com              Cloud Storage API


Let's explore the data and see what tables are there in StackOverflow dataset.

In [ ]:

QUERY_TABLES = """
SELECT
  table_name
FROM
  `bigquery-public-data.stackoverflow.INFORMATION_SCHEMA.TABLES`
"""
print(QUERY_TABLES)
query_job = bq_client.query(QUERY_TABLES)
for row in query_job:
  for value in row.values():
    print(value)


SELECT
  table_name
FROM
  `bigquery-public-data.stackoverflow.INFORMATION_SCHEMA.TABLES`

posts_answers
users
posts_orphaned_tag_wiki
posts_tag_wiki
stackoverflow_posts
posts_questions
comments
posts_tag_wiki_excerpt
posts_wiki_placeholder
posts_privilege_wiki
post_history
badges
post_links
tags
votes
posts_moderator_nomination


### Query optimization and joining tables
We want to do parameter-efficient fine-tuning. Since the data is too big, we export the result of join of python questions and answers into a cloud storage bucket. This enables us to access data quickly.

In [ ]:
QUERY = """
SELECT
  CONCAT(q.title,q.body) AS input_text,
  a.body AS output_text
FROM
  `bigquery-public-data.stackoverflow.posts_questions` q
JOIN
  `bigquery-public-data.stackoverflow.posts_answers` a
ON
  q.accepted_answer_id = a.id
WHERE
  q.accepted_answer_id IS NOT NULL AND
  REGEXP_CONTAINS(q.tags, "python") AND
  a.creation_date >= "2022-01-01"
LIMIT
  10000
"""

query_job = bq_client.query(QUERY)


Let's set the format of the data to be easy for pandas to read.

In [ ]:
import pandas as pd

try:
  stack_overflow_df = query_job\
    .result()\
    .to_arrow()\
    .to_pandas()
except Exception as e:
  print('The DataFrame to loo large to load into memory.',e)
stack_overflow_df.head(5)

,input_text,output_text
0,Using nbconvert to hide all input cells?<p>I h...,<p>You can pass the argument <code>--no-input<...
1,How to multiply each digit of an array in the ...,"<p>if you want a simple solution, use numpy:</..."
2,How to get the text of the message to which th...,"<p>Yes, <a href=""https://discordpy.readthedocs..."
3,How to append datafrarme columns in list?<p>I ...,"<p>Use <a href=""http://pandas.pydata.org/panda..."
4,How to create a custom timerange and convert i...,<p>You could use <code>strftime</code>:</p>\n<...


##Adding Instructions
I create an instruction template for the LLM

In [ ]:
INSTRUCTION_TEMPLATE = f"""\
Please answer the following Stackoverflow question on Python. \
Answer it like you are a developer answering Stackoverflow questions.

Stackoverflow question:
"""

Create an extra column in the table for instructions

In [ ]:
stack_overflow_df['input_text_instruct'] = INSTRUCTION_TEMPLATE + stack_overflow_df['input_text']
stack_overflow_df.head(5)

,input_text,output_text,input_text_instruct
0,Using nbconvert to hide all input cells?<p>I h...,<p>You can pass the argument <code>--no-input<...,Please answer the following Stackoverflow ques...
1,How to multiply each digit of an array in the ...,"<p>if you want a simple solution, use numpy:</...",Please answer the following Stackoverflow ques...
2,How to get the text of the message to which th...,"<p>Yes, <a href=""https://discordpy.readthedocs...",Please answer the following Stackoverflow ques...
3,How to append datafrarme columns in list?<p>I ...,"<p>Use <a href=""http://pandas.pydata.org/panda...",Please answer the following Stackoverflow ques...
4,How to create a custom timerange and convert i...,<p>You could use <code>strftime</code>:</p>\n<...,Please answer the following Stackoverflow ques...


## Dataset for tuning
Now it's time to setup some evaluation set. We're not gonna calculate accuracy because I'm working with text and accuracy is very ambigous on text.

In [ ]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(stack_overflow_df, test_size=0.2, random_state=42)

## Versioning

I'm creating a local versioning system using dated json files

In [ ]:
import datetime

date = datetime.datetime.now().strftime("%Y:%m:%d:%H")
cols = ['input_text_instruct','output_text']
tune_jsonl = train_df[cols].to_json(orient='records', lines=True)
training_data_filename = f"tune_data_stack_overflow_qa-{date}.jsonl"

with open(training_data_filename, 'w') as f:
  f.write(tune_jsonl)

## Orchestration and Automation of a Supervised Tuning Pipeline
The goal at this point is to train the model and evaluate it. Here I use Kubeflow Pipeline which is an open source framework to orchestrate and automate my workflow.

Orchestration explains the series of steps and automation automizes this flow.

In [ ]:
from kfp import dsl, compiler

### Build the pipeline
Kubeflow pipelines consist of Components and pipelines. DSL (domain specific language) is the language used for designing pipelines.
Components run in a containerized environment. We need to connect the componenets to each other. The output from one will be the input to the next one.

Here we use an existing Kubeflow PipeLine for parameter-efficient fine-tuning (PEFT) from Google, called [PaLM 2](https://ai.google/discover/palm2/). Then,
VertexAI manages the Pipeline yaml file in a serverless environment.

It's quite expensive to run the following pipeline and it might take up to one day!

Note: to be able to run the below code, we need to
* Enable Vertex AI API
* give the user this IAM role: Vertex AI Admin
* create buckets

check if the project has access to gemini fine tuning.

In [ ]:
!gcloud beta ai models list --region=us-central1 --project=zeinab-llmops-vertexai-452213


Using endpoint [https://us-central1-aiplatform.googleapis.com/]
Listed 0 items.


If the output of the above command was empty, enable Gemini tuning manually

In [ ]:
!gcloud services enable generativelanguage.googleapis.com --project=zeinab-llmops-vertexai-452213

!gcloud beta ai models list --region=us-central1 --project=zeinab-llmops-vertexai-452213


Using endpoint [https://us-central1-aiplatform.googleapis.com/]
Listed 0 items.


In [ ]:
from google.cloud.aiplatform import PipelineJob
# Define the required configurations
MODEL_NAME = "gemini-1.0"  # You can specify the desired Gemini model version
TRAINING_DATA_URI = "gs://zeinab-llmops-vertexai/training_data.jsonl"  # GCS URI to your training data
EVALUATION_DATA_URI = "gs://zeinab-llmops-vertexai/evaluation_data.jsonl"  # GCS URI to evaluation data
TRAINING_STEPS = 200  # Number of training steps (adjust as needed)
EVALUATION_INTERVAL = 20  # Evaluation frequency

# Get the current date for model naming
date = datetime.datetime.now().strftime("%Y-%m-%d")
MODEL_NAME = f"qa-model-{date}"

# Pipeline arguments
pipeline_arguments = {
    "model_display_name": MODEL_NAME,
    "location": REGION,
    "project": PROJECT_ID,
    "large_model_reference": "gemini-1.0-pro-002",  # Model reference for question answering
    # "base_model_version_id": "textembedding-gecko@003",
    "train_steps": TRAINING_STEPS,
    "dataset_uri": TRAINING_DATA_URI,
    "evaluation_interval": EVALUATION_INTERVAL,
    "evaluation_data_uri": EVALUATION_DATA_URI,
}

# Define the pipeline template path for model tuning
template_path = "https://us-kfp.pkg.dev/ml-pipeline/large-language-model-pipelines/tune-large-model/v2.0.0"

# Create a pipeline job for fine-tuning the model
job = aiplatform.PipelineJob(
    template_path=template_path,
    display_name="fine-tune-gemini-qa",
    parameter_values=pipeline_arguments,
    location=REGION,
    pipeline_root="gs://zeinab-llmops-vertexai/pipeline_root",  # GCS path for storing pipeline artifacts
    enable_caching=True
)

# Submit the pipeline job
job.submit()


FailedPrecondition: 400 Bison model tuning is deprecated. Please migrate to Gemini tuning.

## Predictions, Prompts and Safety
Now that we've obtained the trained model, we want to use it. Here we have to choices, either Batch or REST API. The latter was used in the pipeline.

REST API deploys the model as an api and accesses it like a server. So this is online as opposed to the batch method which is done offline. For REST API therefore, we need low latency. With FAST API or FLASK we can deploy a model as an API and package it in a container. Then we call the model and get a prediction.

In [ ]:
from vertexai.language_models import TextGenerationModel

In [ ]:
model = TextGenerationModel.from_pretrained("text-bison@001")
list_tuned_models = model.list_tuned_model_names()

for i in list_tuned_models:
    print(i)

We see above that we have three instaces of the model, so we can distribute the load between them.